# Evaluate spherical tensor products and transition amplitudes

In [1]:
using JAC, SymEngine

**Note**: The Julia package `SymEngine` is needed to perform symbolic simplifications of spherical tensor products, transition amplitudes, etc. in JAC. By default, this package is not automatically loaded since these symbolic evaluations are a ratjer specific application of JAC:

Spherical tensors are a *key concept* in atomic and nuclear (structure) theory and are central to most computations. While the daily use of JAC does not require much prior knowledge about these tensors, they often occur in the formulation, decomposition and simplification of matrix elements and transition amplitudes, and both at the single-, two- and many-electron formulation of the theory. The use of spherical tensor representations of atomic states, operators and density matrices plays an crucial role as they enable to decompose and simplify many expressions prior to their computations by using the theory of the SU$_2$ special-unitary and SO$_3$ rotation group. It is also very closely related to the use and handling of expressions from Racah's algebra.

In practice, the main goal in dealing with spherical tensors is the (symbolic) manipulation and simplification of products, matrix elements and (transition) amplitudes in spherical tensor notation. This includes (i) the separation of constant operators, external (field) operators and electronic operators that act upon the quantum states; (ii) the *reduction* of matrix elements and amplitudes towards (reduced electronic) matrix elements; (iii) the decompositon of matrix elements that refer to the spin and spatial parts of the associated Hilbert space; and (iv) the representation of many-particle amplitudes into reduced one- and two-particle matrix elements due to the one- and two-particle structure of most, if not all, physical interactions in Nature. To deal with these goals, different representations and distinctions of spherical tensors and tensor components are needed and are (partly) supported by the JAC program. To our knowledge, no other software exist that seriously supports the evaluation and simplification of matrix elements in spherical tensor representation. Apart from atomic structure and collision theory. these developements will be useful for nuclear-structure theory, atomic QED, crystalllography and, perhaps, even for astrophysics.

To make use of symbolic definitions and manipulations, let us start by defining some symbols, similar as for the tutorials on Racah's algebra. We shall use (the global definition of) these symbols below:

In [2]:
j1  = Basic(:j1);    j2  = Basic(:j2);    j3  = Basic(:j3);    j4  = Basic(:j4);    j5  = Basic(:j5)
m1  = Basic(:m1);    m2  = Basic(:m2);    m3  = Basic(:m3);    m4  = Basic(:m4);    m5  = Basic(:m5)
ja  = Basic(:ja);    jb  = Basic(:jb);    jc  = Basic(:jc);    jd  = Basic(:jd);    je  = Basic(:je)
ma  = Basic(:ma);    mb  = Basic(:mb);    mc  = Basic(:mc);    md  = Basic(:md);    me  = Basic(:me)
k   = Basic(:k);     q   = Basic(:q );    K   = Basic(:K);     Q   = Basic(:Q);
kk  = Basic(:kk);    qq  = Basic(:qq);    KK  = Basic(:KK);    QQ  = Basic(:QQ)

QQ

To distinguish different classes of operators, a number (and hierarchy) of **abstract** data types are defined in JAC to represent *particle (electronic)*, *external field* and constant operators as they occur frequently in the decompositions. For example:

In [3]:
? SphericalTensor.AbstractSphericalTensor

`abstract type SphericalTensor.AbstractSphericalTensor`      ... defines an abstract type to comprise various spherical (electron) operators and spherical fields.


It comprises tensors of (abstract) type:

In [4]:
? SphericalTensor.AbstractSphericalOperator

`abstract type SphericalTensor.AbstractSphericalOperator`      ... defines an abstract type to comprise various spherical (electron) operators.

```
+ CkOperator                ... to represent a C^(K) tensor operator.
+ CoulombOperator           ... to represent a V^(0, Coulomb) tensor operator.
+ DipoleOperator            ... to represent a D^(1) dipole operator.
+ MultipoleOperator         ... to represent a O^(L, multipole) operator.
+ TkOperator                ... to represent a general T^(K) tensor operator.
```


In [5]:
? SphericalTensor.AbstractSphericalField

`abstract type SphericalTensor.AbstractSphericalField  <:  AbstractSphericalTensor`      ... defines an abstract type to comprise various (external and electron-independent) field operators.

```
+ UVector                   ... to represent a u^(1) polarization (unit) vector.
+ TwistedU                  ... to represent a u^(1, twisted) unit vector.
```


In [6]:
? SphericalTensor.AbstractSphericalConstant

`abstract type SphericalTensor.AbstractSphericalConstant  <:  AbstractSphericalTensor`      ... defines an abstract type to comprise various (parametric-dependent) scalar constants, such as field amplitudes, etc.

```
+ ScalarConstant            ... to represent a scalar constant to comprise all all factors that do not
                                affect the tensorial structure of matrix elements and amplitues.
```


In [7]:
? SphericalTensor.TensorProduct

`struct  SphericalTensor.TensorProduct  <:  AbstractSphericalTensor`      ... defines the coupled tensor product of two spherical tensors [T1^(k1) * T2^(k2)]^(k).

```
+ star          ::Bool      ... defines the complex-conjugate form, if true. 
+ k             ::Basic     ... (total) tensor rank k.
+ tensor1       ::AbstractSphericalTensor  ... spherical tensor T1.
+ tensor2       ::AbstractSphericalTensor  ... spherical tensor T2.
```

---

`SphericalTensor.TensorProduct(k::Basic, tensor1::AbstractSphericalTensor, tensor2::AbstractSphericalTensor)`       ... constructor for just setting the tensor product (star = false).


The data struct's of these (four) type definitions enables one to define tensors of successive complexity. For example, a simple T^(K) spherical and the rank-1 dipole operators are defined by:

In [8]:
Tk = SphericalTensor.TkOperator(k)

T^(k) spherical tensor operator.


In [9]:
Dop = SphericalTensor.DipoleOperator(false)

D^(1) spherical dipole operator.


Here, the `false` indicates that we do not request for the complex-conjugate (adjoint) dipole operator. Similarly, we can define a (parametrically-dependent) scalar and the polarization (rank-1 vector) of an external plane-wave field by:

In [10]:
g = SphericalTensor.ScalarConstant(false, Basic(:g))

g scalar constant.


In [11]:
uVector = SphericalTensor.UVector(false)

u^(1) spherical polarization (unit) vector.


We can use these pre-defined spherical operators to define tensor products of different rank:

In [12]:
uDopScalar = SphericalTensor.TensorProduct(false, Basic(0), uVector, Dop)

[u^(1) * D^(1)]^(0) spherical tensor product.


In [13]:
uDopVector = SphericalTensor.TensorProduct(false, Basic(1), uVector, Dop)

[u^(1) * D^(1)]^(1) spherical tensor product.


In [14]:
uDopVectorTk = SphericalTensor.TensorProduct(false, K, uDopVector, Tk)

[[u^(1) * D^(1)]^(1) * T^(k)]^(K) spherical tensor product.


Often, however, one starts from matrix elements and expressions that include well-defined tensor components and, hence, from the expansion of such tensor components:

In [15]:
? SphericalTensor.expandSphericalTensorComponent

`SphericalTensor.expandSphericalTensorComponent(tn::SphericalTensor.AbstractSphericalTensorComp, q::Basic)`       ... expand a coupled product of spherical tensors into its (uncoupled) tensor components by means of a Clebsch-Gordan expansion.         All necessary magnetic quantum numbers are 'derived' from the corresponding tensor ranks: K -> qK, 2 -> q2,  0 -> 0.         No expansion is made for elementary spherical tensors. The tensor components of this expansion is returned by (the tuple of)         three variables: (rex::RacahExpression, fields::Array{SphericalTensor.AbstractSphericalField,1},                            operators::Array{SphericalTensor.AbstractSphericalOperator,1}).


which enables us to distinguish and classify between external operators and those that act upon the (electronic) particle states:

In [16]:
SphericalTensor.expandSphericalTensorComponent(uVector, q)

(1, JAC.SphericalTensor.AbstractSphericalConstant[], JAC.SphericalTensor.SphericalFieldComp[u^(1)_q spherical field component.
], JAC.SphericalTensor.SphericalOperatorComp[])

In [17]:
SphericalTensor.expandSphericalTensorComponent(uDopVectorTk, q)

(Sum_[Basic[q1_89, q1_15, q1_84, qk_79]]  (-1)^(1 - k - q1_46 - qK_46)  ((1/3)*sqrt(3)/sqrt(1 + 2*K))  W3j(1, 1, 1; q1_89, q1_15, -q1_46)  W3j(1, k, K; q1_84, qk_79, -qK_46)  , JAC.SphericalTensor.AbstractSphericalConstant[], JAC.SphericalTensor.SphericalFieldComp[u^(1)_q1_89 spherical field component.
], JAC.SphericalTensor.SphericalOperatorComp[D^(1)_q1_15 spherical operator component.
, T^(k)_qk_79 spherical operator component.
])

This function expands the vector component into its product terms by using a Clebsch-Gordan expansion for each individual tensor product. Here, the `magnetic` $q$ quantum numbers are chosen automatically and a random (two-digit) suffix is chosen to make these quantum numbers *unique*.

Apart from the tensors, we need to specify the *spherical* states, i.e. states in angular-momentum basis:

In [18]:
jma = SphericalTensor.SphericalState(false, ja, ma)
jmb = SphericalTensor.SphericalState(false, jb, mb)

|jb mb> spherical state.


We can use these states to form *spherical* matrix elements in which both, the state and the tensor operators are given in spherical representation:

In [19]:
me = SphericalTensor.SphericalMatrixElement(jma, uDopScalar, Basic(0), jmb)

<ja ma | [u^(1) * D^(1)]^(0)_0 | jb mb> reduced matrix element.


and to expand the amplitutes that can be formed by these matrix elements:

In [20]:
meStar = SphericalTensor.SphericalMatrixElement(true, jma, g, uDopScalar, Basic(0), jmb)

<ja ma | g[u^(1) * D^(1)]^(0)_0 | jb mb>^(*) reduced matrix element.


In [21]:
SphericalTensor.expandSphericalMatrixElements([meStar, me])

wd = JAC.SphericalTensor.SphericalOperatorComp[D^(1*)_q1_93 spherical operator component.
]
wd = JAC.SphericalTensor.SphericalOperatorComp[D^(1)_q1_53 spherical operator component.
]


Spherical amplitude with: 
RacahExpression:    Sum_[Basic[q1_93, q1_59, q1_97, q1_53]]  W3j(1, 1, 0; q1_93, q1_59, 0)  W3j(1, 1, 0; q1_97, q1_53, 0)  
Constants:          g^(*)
Field components:   u^(1*)_q1_59  u^(1)_q1_97
productMes:         <jb mb |   D^(1*)_q1_93 | ja ma>   <ja ma |   D^(1)_q1_53 | jb mb> 


# **The following part of this nootebook is still under construction.**
What can be seen from this output ?? .. and what else could be controlled ?? ... for this, we refer the user to the manual.
